# ARPES Spectral Clustering - Methodology Pipeline
## A pipeline for data visualization of 2D mappings using PCA for data reduction and spectral clustering.
#### Authors: Marc Raventós (mraventos@cells.es), Joaquín Otón (joton@cells.es)


In [ ]:
# Location of the .nxs file to process
file_location = ''

In [ ]:
import os

import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn import decomposition
from sklearn.cluster import SpectralClustering

import plotly.graph_objects as go
from plotly.offline import plot

In [ ]:
working_directory = os.getcwd()
file_name = os.path.basename(file_location).rstrip(".nxs")
output_directory = os.path.join(working_directory,f'output_{file_name}') 
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

file = h5py.File(file_location)
data = file['entry1/data/data'][...]

realdimx, realdimy, arpesdimx, arpesdimy = data.shape

In [ ]:
flattened_real = data.reshape(-1,arpesdimx,arpesdimy)
flattened_arpes =  data.reshape(realdimx,realdimy,-1)
flattened_all = flattened_arpes.reshape(realdimx*realdimy,-1)

In [ ]:
data.shape

In [ ]:
std_image = flattened_real.std(0)
fig, ax = plt.subplots(figsize=(9, 9))
im = ax.imshow(std_image,vmax=std_image.max()*0.4)
ax.set_title('Standard deviation image: ARPES space')
ax.set_xticks([]), ax.set_yticks([]) 
# Create a divider for the axis to align the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Add colorbar to the aligned axis
cbar = fig.colorbar(im, cax=cax)
cbar.ax.set_ylabel('Standard Deviation')

plt.tight_layout(pad=0.5)
plt.savefig(os.path.join(output_directory,'std_deviation_arpes_space.png'),dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
std_image = flattened_arpes.std(-1)
fig, ax = plt.subplots(figsize=(9, 9))
im = ax.imshow(std_image.T)
ax.set_xticks([]), ax.set_yticks([]) 
ax.set_title('Standard deviation image: Real space')

# Create a divider for the axis to align the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Add colorbar to the aligned axis
cbar = fig.colorbar(im, cax=cax)
cbar.ax.set_ylabel('Standard Deviation')

plt.tight_layout(pad=0.5)
plt.savefig(os.path.join(output_directory,'std_deviation_real_space.png'),dpi=600, bbox_inches='tight')
plt.show()


We define 2 special interest regions from the data so we can analyze later the result of the clustering

In [ ]:
data.shape

In [ ]:
pca = decomposition.PCA(n_components=200)
X_pca = pca.fit_transform(flattened_all)

# Plot cumulative explained variance
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
         np.cumsum(pca.explained_variance_ratio_),
         marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1))  # Ensure integer x-axis ticks
plt.grid(True)

# Save the figure
plt.savefig(os.path.join(output_directory,'pca_variance.png'))
plt.show()

X_pca.shape

In [ ]:
# Number of components to visualize
n = 3  # Change this to the number of components you want to visualize

# Define subplot layout
n_cols = 3
n_rows = (n + n_cols - 1) // n_cols  # Calculate rows needed for n components

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(9, 9 * n_rows))
axes = axes.flatten()

for i in range(n):
    # Reshape each component to 21x12
    component_image = X_pca[:, i].reshape(realdimx,realdimy).T
    
    # Plot the component
    im = axes[i].imshow(component_image, cmap='viridis')
    axes[i].set_title(f'Principal Component {i + 1}')
    axes[i].axis('off')
    
    # Add a colorbar
    fig.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

# Hide any unused subplots
for i in range(n, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
clustering = SpectralClustering(n_clusters=6,affinity='nearest_neighbors',assign_labels='cluster_qr')
labels_row = clustering.fit_predict(X_pca[:, :3])
sorted_labels = np.argsort([X_pca[labels_row == l].mean(axis=0)[0] for l in np.unique(labels_row)])
label_mapping = {old: new for new, old in enumerate(np.unique(labels_row)[sorted_labels])}
labels = np.vectorize(label_mapping.get)(labels_row)


labels_image = labels.reshape(realdimx, realdimy)


In [ ]:
# Standard deviation image
std_image = flattened_arpes.std(-1)
fig, axes = plt.subplots(2, 1, figsize=(9, 9))  # Two rows, one column

# Plot the standard deviation image
ax1 = axes[0]
im1 = ax1.imshow(std_image.T, cmap='viridis')  # You can customize the colormap
ax1.set_xticks([]), ax1.set_yticks([]) 
ax1.set_title('Standard Deviation Image: Real Space')

# Create a divider for the axis to align the colorbar
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.1)

# Add colorbar to the aligned axis
cbar1 = fig.colorbar(im1, cax=cax1)
cbar1.ax.set_ylabel('Standard Deviation')

# Plot the labels image
ax2 = axes[1]
im2 = ax2.imshow(labels_image.T)  # Customize colormap for categorical data
ax2.set_title('Cluster Labels')
ax2.set_xticks([]), ax2.set_yticks([]) 
# Create a divider for the axis to align the colorbar
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.1)

# Add colorbar to the aligned axis
cbar2 = fig.colorbar(im2, cax=cax2)
cbar2.ax.set_ylabel('Cluster Label')

# Adjust layout
plt.tight_layout(pad=2.0)
plt.savefig(os.path.join(output_directory,'combined_std_and_labels_with_colorbars.png'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# Create a 3D scatter plot with color based on cluster labels
fig = go.Figure(data=[go.Scatter3d(
    x=X_pca[:, 0], 
    y=X_pca[:, 1], 
    z=X_pca[:, 2], 
    mode='markers',
    marker=dict(
        size=5,
        color=labels,  # Color based on cluster labels
        colorscale='Viridis',  # Choose a colorscale
        opacity=0.8,
        colorbar=dict(title="Cluster Label")  # Add colorbar title
    )
)])

# Add axis labels
fig.update_layout(
    scene=dict(
        xaxis_title='Principal Component 1',
        yaxis_title='Principal Component 2',
        zaxis_title='Principal Component 3'
    ),
    title='3D PCA Scatter Plot',
    margin=dict(l=0, r=0, b=0, t=40)  # Adjust margins for better layout
)

# Plot the figure in a separate window
plot(fig, auto_open=True, filename=os.path.join(output_directory,'3d_scatter_plot.html'))


In [ ]:
plt.figure(figsize=(9, 9))
plt.suptitle('Summed up signals of the clusters')

for i in range(labels.max() + 1):
    plt.subplot(3, 2, i + 1)
    image_to_plot = data[labels_image == i, :, :].sum(axis=0)
    plt.imshow(image_to_plot,vmax=image_to_plot.max()*0.4)
    plt.title(f'Cluster label {i}')
    
    # Remove x and y ticks
    plt.xticks([])
    plt.yticks([])

# Adjust layout after all plots are created
plt.tight_layout()

# Save figure once, not inside the loop
plt.savefig(os.path.join(output_directory,'cluster_label_signals.png'), dpi=600, bbox_inches='tight')

plt.show()
